Noisy Deep Ensembleに関する実験用ソースコード

・各種インポート

・GPU接続

・データセット/データローダー

・モデル定義（ResNet18はCIFARに最適化、VGG16とEfficientNetB0はtorchvisionの実装）

・学習/推論の関数定義

・ノイズ付与の関数定義

・実験

の順になっています。


In [4]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
# 各種インポート。使わないものもあるかもしれませんが全部載せておきます。
import torch
import torch.nn as nn
from torch import Tensor
import numpy as np
import pandas as pd
import random
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import statistics
import copy
import seaborn as sns
import time
import torchvision
from collections import OrderedDict
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from numpy.lib.stride_tricks import sliding_window_view
from torchvision import models
from torchinfo import summary
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union
from torchvision.transforms._presets import ImageClassification
from torchvision.utils import _log_api_usage_once
from torchvision.models._api import register_model, Weights, WeightsEnum
from torchvision.models._meta import _IMAGENET_CATEGORIES
from torchvision.models._utils import _ovewrite_named_param, handle_legacy_interface

In [6]:
# gpuに接続
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [7]:
# データセットの読み込み(ResNet18用)
# CIFAR10にデータ拡張は適用していない。

num_batch = 64

# CIFAR10
transform_cifar10_resnet18 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]
    )
])

train_dataset_cifar10_resnet18 = datasets.CIFAR10(
    './CIFAR10_dataset',
    train=True,
    download=True,
    transform=transform_cifar10_resnet18
)

test_dataset_cifar10_resnet18 = datasets.CIFAR10(
    './CIFAR10_dataset',
    train=False,
    download=True,
    transform=transform_cifar10_resnet18
)

train_dataloader_cifar10_resnet18 = DataLoader(
    train_dataset_cifar10_resnet18,
    batch_size=num_batch,
    shuffle=True
)

test_dataloader_cifar10_resnet18 = DataLoader(
    test_dataset_cifar10_resnet18,
    batch_size=num_batch,
    shuffle=True
)


#CIFAR100
transform_cifar100_resnet18 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]
    ),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=[-15,15])
])

transform_cifar100_resnet18_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]
    ),
])

train_dataset_cifar100_resnet18 = datasets.CIFAR100(
    './CIFAR100_dataset',
    train=True,
    download=True,
    transform=transform_cifar100_resnet18
)

test_dataset_cifar100_resnet18 = datasets.CIFAR100(
    './CIFAR100_dataset',
    train=False,
    download=True,
    transform=transform_cifar100_resnet18_test
)

train_dataloader_cifar100_resnet18 = DataLoader(
    train_dataset_cifar100_resnet18,
    batch_size=num_batch,
    shuffle=True
)

test_dataloader_cifar100_resnet18 = DataLoader(
    test_dataset_cifar100_resnet18,
    batch_size=num_batch,
    shuffle=True
)

cifar10_resnet18 = [train_dataloader_cifar10_resnet18, test_dataloader_cifar10_resnet18]
cifar100_resnet18 = [train_dataloader_cifar100_resnet18, test_dataloader_cifar100_resnet18]

100%|██████████| 170498071/170498071 [00:11<00:00, 14962655.15it/s]


Extracting ./CIFAR10_dataset/cifar-10-python.tar.gz to ./CIFAR10_dataset
Files already downloaded and verified


100%|██████████| 169001437/169001437 [00:10<00:00, 15441975.37it/s]


Extracting ./CIFAR100_dataset/cifar-100-python.tar.gz to ./CIFAR100_dataset
Files already downloaded and verified


In [8]:
#データセットの読み込み(vgg16用)

num_batch = 64

#CIFAR10
transform_cifar10_vgg16 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    ),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=[-15,15])
])

transform_cifar10_vgg16_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

train_dataset_cifar10_vgg16 = datasets.CIFAR10(
    './CIFAR10_dataset',
    train=True,
    download=True,
    transform=transform_cifar10_vgg16
)

test_dataset_cifar10_vgg16 = datasets.CIFAR10(
    './CIFAR10_dataset',
    train=False,
    download=True,
    transform=transform_cifar10_vgg16_test
)

train_dataloader_cifar10_vgg16 = DataLoader(
    train_dataset_cifar10_vgg16,
    batch_size=num_batch,
    shuffle=True
)

test_dataloader_cifar10_vgg16 = DataLoader(
    test_dataset_cifar10_vgg16,
    batch_size=num_batch,
    shuffle=True
)


#CIFAR100
transform_cifar100_vgg16 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]
    ),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=[-15,15])
])

transform_cifar100_vgg16_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]
    )
])

train_dataset_cifar100_vgg16 = datasets.CIFAR100(
    './CIFAR100_dataset',
    train=True,
    download=True,
    transform=transform_cifar100_vgg16
)

test_dataset_cifar100_vgg16 = datasets.CIFAR100(
    './CIFAR100_dataset',
    train=False,
    download=True,
    transform=transform_cifar100_vgg16_test
)

train_dataloader_cifar100_vgg16 = DataLoader(
    train_dataset_cifar100_vgg16,
    batch_size=num_batch,
    shuffle=True
)

test_dataloader_cifar100_vgg16 = DataLoader(
    test_dataset_cifar100_vgg16,
    batch_size=num_batch,
    shuffle=True
)

cifar10_vgg16 = [train_dataloader_cifar10_vgg16, test_dataloader_cifar10_vgg16]
cifar100_vgg16 = [train_dataloader_cifar100_vgg16, test_dataloader_cifar100_vgg16]


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [23]:
# データセットの読み込み(EfficientNetB0用)
# B0に入力する画像のサイズは224x224が適しているらしいので、ここで32x32→224x224に変換している。

num_batch = 512

# CIFAR10

transform_cifar10_efficient = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224, 224), antialias=True),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    ),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=[-15,15])
])

transform_cifar10_efficient_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224, 224), antialias=True),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

train_dataset_cifar10_efficient = datasets.CIFAR10(
    './CIFAR10_dataset',
    train=True,
    download=True,
    transform=transform_cifar10_efficient
)

test_dataset_cifar10_efficient = datasets.CIFAR10(
    './CIFAR10_dataset',
    train=False,
    download=True,
    transform=transform_cifar10_efficient_test
)

train_dataloader_cifar10_efficient = DataLoader(
    train_dataset_cifar10_efficient,
    batch_size=num_batch,
    shuffle=True
)

test_dataloader_cifar10_efficient = DataLoader(
    test_dataset_cifar10_efficient,
    batch_size=num_batch,
    shuffle=True
)


# CIFAR100

transform_cifar100_efficient = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224, 224), antialias=True),
    transforms.Normalize(
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]
    ),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=[-15,15])
])

transform_cifar100_efficient_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224, 224), antialias=True),
    transforms.Normalize(
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]
    )
])

train_dataset_cifar100_efficient = datasets.CIFAR100(
    './CIFAR100_dataset',
    train=True,
    download=True,
    transform=transform_cifar100_efficient
)

test_dataset_cifar100_efficient = datasets.CIFAR100(
    './CIFAR100_dataset',
    train=False,
    download=True,
    transform=transform_cifar100_efficient_test
)

train_dataloader_cifar100_efficient = DataLoader(
    train_dataset_cifar100_efficient,
    batch_size=num_batch,
    shuffle=True
)

test_dataloader_cifar100_efficient = DataLoader(
    test_dataset_cifar100_efficient,
    batch_size=num_batch,
    shuffle=True
)

cifar10_efficient = [train_dataloader_cifar10_efficient, test_dataloader_cifar10_efficient]
cifar100_efficient = [train_dataloader_cifar100_efficient, test_dataloader_cifar100_efficient]

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [24]:
# ResNet18
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )

def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition" https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        _log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        #--------------------------------------------------------------------------------------------------------------------------------------------
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, padding=1, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        #--------------------------------------------------------------------------------------------------------------------------------------------
        # x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    weights: Optional[WeightsEnum],
    progress: bool,
    **kwargs: Any,
) -> ResNet:
    if weights is not None:
        _ovewrite_named_param(kwargs, "num_classes", len(weights.meta["categories"]))

    model = ResNet(block, layers, **kwargs)

    if weights is not None:
        model.load_state_dict(weights.get_state_dict(progress=progress))

    return model


_COMMON_META = {
    "min_size": (1, 1),
    "categories": _IMAGENET_CATEGORIES,
}

def resnet18(num_classes):
    model = ResNet(BasicBlock, [2,2,2,2], num_classes=num_classes)
    return model


In [25]:
# VGG16
def vgg16(num_classes):
    model = models.vgg16(num_classes=num_classes)
    return model

In [26]:
# EfficientNetB0
def efficientnetb0(num_classes):
    model = models.efficientnet_b0(num_classes=num_classes)


In [27]:
def model_train(model, dataset, epochs, lr, log=True, acc=False):
    log_train_acc = []
    log_train_loss = []
    log_test_acc = []
    log_test_loss = []

    model.to(device)

    # SGD　損失関数　学習率スケジューラの設定
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    for epoch in range(epochs):
        train_acc = 0
        train_loss = 0
        test_acc = 0
        test_loss = 0

        model.train()
        for inputs , labels in dataset[0]:
            optimizer.zero_grad()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            train_loss += loss.item()
            train_acc += (outputs.max(1)[1] == labels).sum().item()
            loss.backward()
            optimizer.step()
        
        scheduler.step()
        
        model.eval()
        with torch.no_grad():
            for inputs, labels in dataset[1]:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                test_acc += (outputs.max(1)[1] == labels).sum().item()
                
        log_train_loss.append(train_loss / len(dataset[0]))
        log_test_loss.append(test_loss / len(dataset[1]))
        # Accuracyを計算
        log_train_acc.append(train_acc / len(dataset[0].dataset))
        log_test_acc.append(test_acc / len(dataset[1].dataset))

        if log == True:
            print("Epoch[{}/{}], train_acc:{acc:.4f}, train_loss:{loss:.4f}, test_acc:{test_acc:.4f}, test_loss:{test_loss:.4f}"
                        .format(epoch+1, epochs, acc=log_train_acc[-1], loss=log_train_loss[-1], test_acc=log_test_acc[-1], test_loss=log_test_loss[-1]))
    
    if acc == True:
        return log_test_acc[-1]

In [28]:
def model_test(model, dataset, acc=False):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    log_test_acc = []
    log_test_loss = []
    test_acc = 0
    test_loss = 0

    with torch.no_grad():
        for inputs, labels in dataset[1]:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            test_acc += (outputs.max(1)[1] == labels).sum().item()
        
    log_test_loss.append(test_loss / len(dataset[1]))
    log_test_acc.append(test_acc / len(dataset[1].dataset))

    print("test_acc:{val_acc:.4f}, test_loss:{val_loss:.4f}"
                  .format(val_acc=log_test_acc[-1], val_loss=log_test_loss[-1]))
    
    if acc == True:
        return log_test_acc[-1]
    

In [29]:
def ens_test(model_list, dataset, acc=False):
    criterion = nn.CrossEntropyLoss()
    log_test_acc = []
    log_test_loss = []
    test_acc = 0
    test_loss = 0

    with torch.no_grad():
        for inputs, labels in dataset[1]:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_list[0](inputs)
            for i in range(len(model_list)-1):
                outputs += model_list[i+1](inputs)
            outputs = outputs / len(model_list)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            test_acc += (outputs.max(1)[1] == labels).sum().item()
    
    log_test_loss.append(test_loss / len(dataset[1]))
    log_test_acc.append(test_acc / len(dataset[1].dataset))
    print("test_acc:{val_acc:.4f}, test_loss:{val_loss:.4f}"
                  .format(val_acc=log_test_acc[-1], val_loss=log_test_loss[-1]))
    
    if acc == True:
        return log_test_acc[-1]

In [30]:
# 各モデルアーキテクチャのノイズ付与対象レイヤーのリスト。

resnet18_layer_list = ['conv1.weight', 
              'layer1.0.conv1.weight',
              'layer1.0.conv2.weight',
              'layer1.1.conv1.weight',
              'layer1.1.conv2.weight',

              'layer2.0.conv1.weight',
              'layer2.0.conv2.weight',
              'layer2.1.conv1.weight',
              'layer2.1.conv2.weight',

              'layer3.0.conv1.weight',
              'layer3.0.conv2.weight',
              'layer3.1.conv1.weight',
              'layer3.1.conv2.weight',
              
              'layer4.0.conv1.weight',
              'layer4.0.conv2.weight',
              'layer4.1.conv1.weight',
              'layer4.1.conv2.weight',
              'fc.weight'
              ]

vgg16_layer_list = ['features.0.weight',
                'features.2.weight',
                'features.5.weight',
                'features.7.weight',
                'features.10.weight',
                'features.12.weight',
                'features.14.weight',
                'features.17.weight',
                'features.19.weight',
                'features.21.weight',
                'features.24.weight',
                'features.26.weight',
                'features.28.weight',
                'classifier.0.weight',
                'classifier.3.weight',
                'classifier.6.weight'
                ]

efficientnetb0_layer_list = ['features.0.0.weight',
                'features.0.1.weight',
                'features.1.0.block.0.0.weight',
                'features.1.0.block.0.1.weight',
                'features.1.0.block.1.fc1.weight',
                'features.1.0.block.1.fc2.weight',
                'features.1.0.block.2.0.weight',
                'features.1.0.block.2.1.weight',
                'features.2.0.block.0.0.weight',
                'features.2.0.block.0.1.weight',
                'features.2.0.block.1.0.weight',
                'features.2.0.block.1.1.weight',
                'features.2.0.block.2.fc1.weight',
                'features.2.0.block.2.fc2.weight',
                'features.2.0.block.3.0.weight',
                'features.2.0.block.3.1.weight',
                'features.2.1.block.0.0.weight',
                'features.2.1.block.0.1.weight',
                'features.2.1.block.1.0.weight',
                'features.2.1.block.1.1.weight',
                'features.2.1.block.2.fc1.weight',
                'features.2.1.block.2.fc2.weight',
                'features.2.1.block.3.0.weight',
                'features.2.1.block.3.1.weight',
                'features.3.0.block.0.0.weight',
                'features.3.0.block.0.1.weight',
                'features.3.0.block.1.0.weight',
                'features.3.0.block.1.1.weight',
                'features.3.0.block.2.fc1.weight',
                'features.3.0.block.2.fc2.weight',
                'features.3.0.block.3.0.weight',
                'features.3.0.block.3.1.weight',
                'features.3.1.block.0.0.weight',
                'features.3.1.block.0.1.weight',
                'features.3.1.block.1.0.weight',
                'features.3.1.block.1.1.weight',
                'features.3.1.block.2.fc1.weight',
                'features.3.1.block.2.fc2.weight',
                'features.3.1.block.3.0.weight',
                'features.3.1.block.3.1.weight',
                'features.4.0.block.0.0.weight',
                'features.4.0.block.0.1.weight',
                'features.4.0.block.1.0.weight',
                'features.4.0.block.1.1.weight',
                'features.4.0.block.2.fc1.weight',
                'features.4.0.block.2.fc2.weight',
                'features.4.0.block.3.0.weight',
                'features.4.0.block.3.1.weight',
                'features.4.1.block.0.0.weight',
                'features.4.1.block.0.1.weight',
                'features.4.1.block.1.0.weight',
                'features.4.1.block.1.1.weight',
                'features.4.1.block.2.fc1.weight',
                'features.4.1.block.2.fc2.weight',
                'features.4.1.block.3.0.weight',
                'features.4.1.block.3.1.weight',
                'features.4.2.block.0.0.weight',
                'features.4.2.block.0.1.weight',
                'features.4.2.block.1.0.weight',
                'features.4.2.block.1.1.weight',
                'features.4.2.block.2.fc1.weight',
                'features.4.2.block.2.fc2.weight',
                'features.4.2.block.3.0.weight',
                'features.4.2.block.3.1.weight',
                'features.5.0.block.0.0.weight',
                'features.5.0.block.0.1.weight',
                'features.5.0.block.1.0.weight',
                'features.5.0.block.1.1.weight',
                'features.5.0.block.2.fc1.weight',
                'features.5.0.block.2.fc2.weight',
                'features.5.0.block.3.0.weight',
                'features.5.0.block.3.1.weight',
                'features.5.1.block.0.0.weight',
                'features.5.1.block.0.1.weight',
                'features.5.1.block.1.0.weight',
                'features.5.1.block.1.1.weight',
                'features.5.1.block.2.fc1.weight',
                'features.5.1.block.2.fc2.weight',
                'features.5.1.block.3.0.weight',
                'features.5.1.block.3.1.weight',
                'features.5.2.block.0.0.weight',
                'features.5.2.block.0.1.weight',
                'features.5.2.block.1.0.weight',
                'features.5.2.block.1.1.weight',
                'features.5.2.block.2.fc1.weight',
                'features.5.2.block.2.fc2.weight',
                'features.5.2.block.3.0.weight',
                'features.5.2.block.3.1.weight',
                'features.6.0.block.0.0.weight',
                'features.6.0.block.0.1.weight',
                'features.6.0.block.1.0.weight',
                'features.6.0.block.1.1.weight',
                'features.6.0.block.2.fc1.weight',
                'features.6.0.block.2.fc2.weight',
                'features.6.0.block.3.0.weight',
                'features.6.0.block.3.1.weight',
                'features.6.1.block.0.0.weight',
                'features.6.1.block.0.1.weight',
                'features.6.1.block.1.0.weight',
                'features.6.1.block.1.1.weight',
                'features.6.1.block.2.fc1.weight',
                'features.6.1.block.2.fc2.weight',
                'features.6.1.block.3.0.weight',
                'features.6.1.block.3.1.weight',
                'features.6.2.block.0.0.weight',
                'features.6.2.block.0.1.weight',
                'features.6.2.block.1.0.weight',
                'features.6.2.block.1.1.weight',
                'features.6.2.block.2.fc1.weight',
                'features.6.2.block.2.fc2.weight',
                'features.6.2.block.3.0.weight',
                'features.6.2.block.3.1.weight',
                'features.6.3.block.0.0.weight',
                'features.6.3.block.0.1.weight',
                'features.6.3.block.1.0.weight',
                'features.6.3.block.1.1.weight',
                'features.6.3.block.2.fc1.weight',
                'features.6.3.block.2.fc2.weight',
                'features.6.3.block.3.0.weight',
                'features.6.3.block.3.1.weight',
                'features.7.0.block.0.0.weight',
                'features.7.0.block.0.1.weight',
                'features.7.0.block.1.0.weight',
                'features.7.0.block.1.1.weight',
                'features.7.0.block.2.fc1.weight',
                'features.7.0.block.2.fc2.weight',
                'features.7.0.block.3.0.weight',
                'features.7.0.block.3.1.weight',
                'features.8.0.weight',
                'features.8.1.weight',
                'classifier.1.weight']

In [31]:
# 一様分布ノイズ(uni noise)

def make_uni_noise(weight, a, b):
    w = weight.cpu().detach().numpy().copy()
    size = w.size
    o_shape = w.shape
    w = w.reshape(-1)
    p = np.random.choice(size, int(size*a), replace=False)
    
    for i in range(p.size):
        noise = random.uniform(-b, b)
        tmp = w[p[i]]
        w[p[i]] += noise
        if(tmp * w[p[i]] < 0):
            w[p[i]] *= -1

    w = w.reshape(o_shape)
    new_weight = torch.from_numpy(w).clone()

    return new_weight

def uni_noise(model, model_arc, a, b):
    new_model = copy.deepcopy(model)
    if model_arc == 'resnet18':
        l_list = resnet18_layer_list
    elif model_arc == 'vgg16':
        l_list = vgg16_layer_list
    elif model_arc == 'efficientnetb0':
        l_list = efficientnetb0_layer_list
    else: print('no such a model')

    for l in l_list:
        new_model.state_dict()[l][0:new_model.state_dict()[l].shape[0]] = make_uni_noise(new_model.state_dict()[l], a, b)
    
    return new_model

In [32]:
# 正規分布ノイズ(norm noise)

def make_norm_noise(weight, a, mean, b):
    w = weight.cpu().detach().numpy().copy()
    size = w.size
    o_shape = w.shape
    w = w.reshape(-1)
    p = np.random.choice(size, int(size*a), replace=False)
    
    for i in range(p.size):
        noise = random.normalvariate(mean, b)
        tmp = w[p[i]]
        w[p[i]] += noise
        if(tmp * w[p[i]] < 0):
            w[p[i]] *= -1
    
    w = w.reshape(o_shape)
    new_weight = torch.from_numpy(w).clone()

    return new_weight

def norm_noise(model, model_arc, a, mean, b):
    new_model = copy.deepcopy(model)
    if model_arc == 'resnet18':
        l_list = resnet18_layer_list
    elif model_arc == 'vgg16':
        l_list = vgg16_layer_list
    elif model_arc == 'efficientnetb0':
        l_list = efficientnetb0_layer_list
    else: print('no such a model')

    for l in l_list:
        new_model.state_dict()[l][0:new_model.state_dict()[l].shape[0]] = make_norm_noise(new_model.state_dict()[l], a, mean, b)
    return new_model

実験

#1行目は「ノイズ手法」、「データセット」、「モデルアーキテクチャ」の順に表示。

XXX_acc_list_XXX：M = 1 ~ 10の場合のaccuracyを格納したリスト。

XXX_model_list_XXX：アンサンブルを構成するモデルを格納したリスト。

アンサンブルメンバ数Mは10。

関数uni_noise：一様分布に基づくノイズを付与。引数は左から「ノイズ付与するモデル」、「モデルアーキテクチャ」、「ノイズ付与率α」、「ノイズ強度β」。返り値はモデル。

関数norm_noise：正規分布に基づくノイズを付与。引数は左から「ノイズ付与するモデル」、「モデルアーキテクチャ」、「ノイズ付与率α」、「分布の平均（0固定でよい）」、「ノイズ強度β」。返り値はモデル。

関数model_train：モデルの学習。引数は左から「学習するモデル」、「データセット」、「エポック」、「学習率」、「学習のログ出力の有無」。

関数model_test：単一モデルのテスト。引数は左から「モデル」、「データセット」、「返り値としてaccの値を返すか（True固定でよい）」。

関数ens_test：アンサンブルモデルのテスト。引数は左から「モデルのリスト」、「データセット」、「返り値としてaccの値を返すか（True固定でよい）」。

SGD、損失関数、学習率スケジューラの設定に関しては関数model_trainの定義で
ノイズを伴う手法に関しては付属のファイルから事前学習モデルをロードする必要あり
ハイパーパラメータは本実験で使用したままになっている

In [33]:
# standard ensemble CIFAR10 ResNet18
M = 1
standard_acc_list_c10_r18 = []
standard_model_list_c10_r18 = []

for i in range(M):
    model = resnet18(10)
    model_train(model, cifar10_resnet18, 200, 0.1, log=True)
    standard_model_list_c10_r18.append(model)
    if i == 0:
        acc = model_test(model, cifar10_resnet18, acc=True)
        standard_acc_list_c10_r18.append(acc)
    else:
        acc = ens_test(standard_model_list_c10_r18, cifar10_resnet18, acc=True)
        standard_acc_list_c10_r18.append(acc)

print(standard_acc_list_c10_r18)
    

Epoch[1/200], train_acc:0.3253, train_loss:1.9573, test_acc:0.4116, test_loss:1.6175
Epoch[2/200], train_acc:0.5186, train_loss:1.3248, test_acc:0.5745, test_loss:1.1722
Epoch[3/200], train_acc:0.6491, train_loss:0.9898, test_acc:0.6623, test_loss:0.9900
Epoch[4/200], train_acc:0.7347, train_loss:0.7589, test_acc:0.7425, test_loss:0.7620
Epoch[5/200], train_acc:0.7754, train_loss:0.6502, test_acc:0.7124, test_loss:0.8317
Epoch[6/200], train_acc:0.7959, train_loss:0.5850, test_acc:0.7726, test_loss:0.6704
Epoch[7/200], train_acc:0.8144, train_loss:0.5365, test_acc:0.7637, test_loss:0.6969
Epoch[8/200], train_acc:0.8250, train_loss:0.5051, test_acc:0.7461, test_loss:0.7595
Epoch[9/200], train_acc:0.8358, train_loss:0.4783, test_acc:0.7405, test_loss:0.8240
Epoch[10/200], train_acc:0.8442, train_loss:0.4515, test_acc:0.7763, test_loss:0.6765
Epoch[11/200], train_acc:0.8444, train_loss:0.4481, test_acc:0.7534, test_loss:0.7607
Epoch[12/200], train_acc:0.8518, train_loss:0.4289, test_acc:0.

In [36]:
len(test_dataset_cifar10_resnet18)

10000

In [ ]:
# uni noise CIFAR10 ResNet18
M = 10
uni_acc_list_c10_r18 = []
uni_model_list_c10_r18 = []

for i in range(M):
    model = resnet18(10)
    model.load_state_dict(torch.load()) # model_pretrain_ResNet18_CIFAR10.pth
    model = uni_noise(model, 'resnet18', 0.8, 1.6)
    model_train(model, cifar10_resnet18, 50, 0.1, log=False)
    uni_model_list_c10_r18.append(model)
    if i == 0:
        acc = model_test(model, cifar10_resnet18, acc=True)
        uni_acc_list_c10_r18.append(acc)
    else:
        acc = ens_test(uni_model_list_c10_r18, cifar10_resnet18, acc=True)
        uni_acc_list_c10_r18.append(acc)
        
print(uni_acc_list_c10_r18)

In [ ]:
# norm noise CIFAR10 ResNet18
M = 10
norm_acc_list_c10_r18 = []
norm_model_list_c10_r18 = []

for i in range(M):
    model = resnet18(10)
    model.load_state_dict(torch.load()) # model_pretrain_ResNet18_CIFAR10.pth
    model = norm_noise(model, 'resnet18', 0.8, 0.0, 1.6)
    model_train(model, cifar10_resnet18, 50, 0.1, log=False)
    norm_model_list_c10_r18.append(model)
    if i == 0:
        acc = model_test(model, cifar10_resnet18, acc=True)
        norm_acc_list_c10_r18.append(acc)
    else:
        acc = ens_test(norm_model_list_c10_r18, cifar10_resnet18, acc=True)
        norm_acc_list_c10_r18.append(acc)
        
print(norm_acc_list_c10_r18)

In [ ]:
# standard ensemble CIFAR100 ResNet18
M = 10
standard_acc_list_c100_r18 = []
standard_model_list_c100_r18 = []

for i in range(M):
    model = resnet18(100)
    model_train(model, cifar100_resnet18, 200, 0.1, log=False)
    standard_model_list_c100_r18.append(model)
    if i == 0:
        acc = model_test(model, cifar100_resnet18, acc=True)
        standard_acc_list_c100_r18.append(acc)
    else:
        acc = ens_test(standard_model_list_c100_r18, cifar100_resnet18, acc=True)
        standard_acc_list_c100_r18.append(acc)

print(standard_acc_list_c100_r18)

In [ ]:
# uni noise CIFAR100 ResNet18
M = 10
uni_acc_list_c100_r18 = []
uni_model_list_c100_r18 = []

for i in range(M):
    model = resnet18(100)
    model.load_state_dict(torch.load()) # model_pretrain_ResNet18_CIFAR100.pth
    model = uni_noise(model, 'resnet18', 0.8, 1.6)
    model_train(model, cifar100_resnet18, 50, 0.1, log=False)
    uni_model_list_c100_r18.append(model)
    if i == 0:
        acc = model_test(model, cifar100_resnet18, acc=True)
        uni_acc_list_c100_r18.append(acc)
    else:
        acc = ens_test(uni_model_list_c10_r18, cifar100_resnet18, acc=True)
        uni_acc_list_c100_r18.append(acc)
        
print(uni_acc_list_c100_r18)

In [ ]:
# norm noise CIFAR100 ResNet18
M = 10
norm_acc_list_c100_r18 = []
norm_model_list_c100_r18 = []

for i in range(M):
    model = resnet18(100)
    model.load_state_dict(torch.load()) # model_pretrain_ResNet18_CIFAR100.pth
    model = norm_noise(model, 'resnet18', 0.8, 0.0, 1.6)
    model_train(model, cifar100_resnet18, 50, 0.1, log=False)
    norm_model_list_c100_r18.append(model)
    if i == 0:
        acc = model_test(model, cifar100_resnet18, acc=True)
        norm_acc_list_c100_r18.append(acc)
    else:
        acc = ens_test(norm_model_list_c100_r18, cifar100_resnet18, acc=True)
        norm_acc_list_c100_r18.append(acc)
        
print(norm_acc_list_c100_r18)

In [ ]:
# standard ensemble CIFAR10 VGG16 
M = 10
standard_acc_list_c10_v16 = []
standard_model_list_c10_v16 = []

for i in range(M):
    model = vgg16(10)
    model_train(model, cifar10_vgg16, 200, 0.01, log=False)
    standard_model_list_c10_v16.append(model)
    if i == 0:
        acc = model_test(model, cifar10_vgg16, acc=True)
        standard_acc_list_c10_v16.append(acc)
    else:
        acc = ens_test(standard_model_list_c10_v16, cifar10_vgg16, acc=True)
        standard_acc_list_c10_v16.append(acc)

print(standard_acc_list_c10_v16)

In [ ]:
# uni noise CIFAR10 VGG16
M = 10
uni_acc_list_c10_v16 = []
uni_model_list_c10_v16 = []

for i in range(M):
    model = vgg16(10)
    model.load_state_dict(torch.load()) # model_pretrain_VGG16_CIFAR10.pth
    model = uni_noise(model, 'vgg16', 0.1, 0.1)
    model_train(model, cifar10_vgg16, 50, 0.01, log=False)
    uni_model_list_c10_v16.append(model)
    if i == 0:
        acc = model_test(model, cifar10_vgg16, acc=True)
        uni_acc_list_c10_v16.append(acc)
    else:
        acc = ens_test(uni_model_list_c10_v16, cifar10_vgg16, acc=True)
        uni_acc_list_c10_v16.append(acc)

print(uni_acc_list_c10_v16)

In [ ]:
# norm noise CIFAR10 VGG16
M = 10
norm_acc_list_c10_v16 = []
norm_model_list_c10_v16 = []

for i in range(M):
    model = vgg16(10)
    model.load_state_dict(torch.load()) # model_pretrain_VGG16_CIFAR10.pth
    model = norm_noise(model, 'vgg16', 0.1, 0.0, 0.01)
    model_train(model, cifar10_vgg16, 50, 0.01, log=False)
    norm_model_list_c10_v16.append(model)
    if i == 0:
        acc = model_test(model, cifar10_vgg16, acc=True)
        norm_acc_list_c10_v16.append(acc)
    else:
        acc = ens_test(norm_model_list_c10_v16, cifar10_vgg16, acc=True)
        norm_acc_list_c10_v16.append(acc)

print(norm_acc_list_c10_v16)

In [ ]:
# standard ensemble CIFAR100 VGG16 
M = 10
standard_acc_list_c100_v16 = []
standard_model_list_c100_v16 = []

for i in range(M):
    model = vgg16(100)
    model_train(model, cifar100_vgg16, 200, 0.01, log=False)
    standard_model_list_c100_v16.append(model)
    if i == 0:
        acc = model_test(model, cifar100_vgg16, acc=True)
        standard_acc_list_c100_v16.append(acc)
    else:
        acc = ens_test(standard_model_list_c100_v16, cifar100_vgg16, acc=True)
        standard_acc_list_c100_v16.append(acc)

print(standard_acc_list_c100_v16)

In [ ]:
# uni noise CIFAR100 VGG16
M = 10
uni_acc_list_c100_v16 = []
uni_model_list_c100_v16 = []

for i in range(M):
    model = vgg16(100)
    model.load_state_dict(torch.load()) # model_pretrain_VGG16_CIFAR100.pth
    model = uni_noise(model, 'vgg16', 0.1, 0.1)
    model_train(model, cifar100_vgg16, 50, 0.01, log=False)
    uni_model_list_c100_v16.append(model)
    if i == 0:
        acc = model_test(model, cifar100_vgg16, acc=True)
        uni_acc_list_c100_v16.append(acc)
    else:
        acc = ens_test(uni_model_list_c100_v16, cifar100_vgg16, acc=True)
        uni_acc_list_c100_v16.append(acc)

print(uni_acc_list_c100_v16)

In [ ]:
# norm noise CIFAR100 VGG16
M = 10
norm_acc_list_c100_v16 = []
norm_model_list_c100_v16 = []

for i in range(M):
    model = vgg16(100)
    model.load_state_dict(torch.load()) # model_pretrain_VGG16_CIFAR100.pth
    model = norm_noise(model, 'vgg16', 0.05, 0.0, 0.05)
    model_train(model, cifar100_vgg16, 50, 0.01, log=False)
    norm_model_list_c100_v16.append(model)
    if i == 0:
        acc = model_test(model, cifar100_vgg16, acc=True)
        norm_acc_list_c100_v16.append(acc)
    else:
        acc = ens_test(norm_model_list_c100_v16, cifar100_vgg16, acc=True)
        norm_acc_list_c100_v16.append(acc)

print(norm_acc_list_c100_v16)

In [ ]:
# standard ensemble CIFAR10 EfficientNetB0
M = 10
standard_acc_list_c10_eb0 = []
standard_model_list_c10_eb0 = []

for i in range(M):
    model = efficientnetb0(10)
    model_train(model, cifar10_efficient, 100, 0.1, log=False)
    standard_model_list_c10_eb0.append(model)
    if i == 0:
        acc = model_test(model, cifar10_efficient, acc=True)
        standard_acc_list_c10_eb0.append(acc)
    else:
        acc = ens_test(standard_model_list_c10_eb0, cifar10_efficient, acc=True)
        standard_acc_list_c10_eb0.append(acc)

print(standard_acc_list_c10_eb0)


In [ ]:
# uni noise CIFAR10 EfficientNetB0
M = 10

uni_acc_list_c10_eb0 = []
uni_model_list_c10_eb0 = []

for i in range(M):
    model = efficientnetb0(10)
    model.load_state_dict(torch.load()) # model_pretrain_EfficientNetB0_CIFAR100.pth
    model = uni_noise(model, 'efficientnetb0', 0.7, 0.1)
    model_train(model, cifar10_efficient, 25, 0.1, log=False)
    uni_model_list_c10_eb0.append(model)
    if i == 0:
        acc = model_test(model, cifar10_efficient, acc=True)
        uni_acc_list_c10_eb0.append(acc)
    else:
        acc = ens_test(uni_model_list_c10_eb0, cifar10_efficient, acc=True)
        uni_acc_list_c10_eb0.append(acc)

print(uni_acc_list_c10_eb0)

In [ ]:
# norm noise CIFAR10 EfficientNetB0
M = 10

norm_acc_list_c10_eb0 = []
norm_model_list_c10_eb0 = []

for i in range(M):
    model = efficientnetb0(10)
    model.load_state_dict(torch.load()) # model_pretrain_EfficientNetB0_CIFAR100.pth
    model = norm_noise(model, 'efficientnetb0', 0.3, 0.0, 0.1)
    model_train(model, cifar10_efficient, 25, 0.1, log=False)
    norm_model_list_c10_eb0.append(model)
    if i == 0:
        acc = model_test(model, cifar10_efficient, acc=True)
        norm_acc_list_c10_eb0.append(acc)
    else:
        acc = ens_test(norm_model_list_c10_eb0, cifar10_efficient, acc=True)
        norm_acc_list_c10_eb0.append(acc)

print(norm_acc_list_c10_eb0)

In [ ]:
# standard ensemble CIFAR100 EfficientNetB0
M = 10
standard_acc_list_c100_eb0 = []
standard_model_list_c100_eb0 = []

for i in range(M):
    model = efficientnetb0(100)
    model_train(model, cifar100_efficient, 100, 0.1, log=False)
    standard_model_list_c100_eb0.append(model)
    if i == 0:
        acc = model_test(model, cifar100_efficient, acc=True)
        standard_acc_list_c100_eb0.append(acc)
    else:
        acc = ens_test(standard_model_list_c100_eb0, cifar100_efficient, acc=True)
        standard_acc_list_c100_eb0.append(acc)

print(standard_acc_list_c100_eb0)


In [ ]:
# uni noise CIFAR100 EfficientNetB0
M = 10

uni_acc_list_c100_eb0 = []
uni_model_list_c100_eb0 = []

for i in range(M):
    model = efficientnetb0(100)
    model.load_state_dict(torch.load()) # model_pretrain_EfficientNetB0_CIFAR100.pth
    model = uni_noise(model, 'efficientnetb0', 0.3, 0.3)
    model_train(model, cifar100_efficient, 25, 0.1, log=False)
    uni_model_list_c100_eb0.append(model)
    if i == 0:
        acc = model_test(model, cifar100_efficient, acc=True)
        uni_acc_list_c100_eb0.append(acc)
    else:
        acc = ens_test(uni_model_list_c100_eb0, cifar100_efficient, acc=True)
        uni_acc_list_c100_eb0.append(acc)

print(uni_acc_list_c100_eb0)

In [ ]:
# norm noise CIFAR100 EfficientNetB0
M = 10

norm_acc_list_c100_eb0 = []
norm_model_list_c100_eb0 = []

for i in range(M):
    model = efficientnetb0(100)
    model.load_state_dict(torch.load()) # model_pretrain_EfficientNetB0_CIFAR100.pth
    model = norm_noise(model, 'efficientnetb0', 0.3, 0.0, 0.3)
    model_train(model, cifar100_efficient, 25, 0.1, log=False)
    norm_model_list_c100_eb0.append(model)
    if i == 0:
        acc = model_test(model, cifar100_efficient, acc=True)
        norm_acc_list_c100_eb0.append(acc)
    else:
        acc = ens_test(norm_model_list_c100_eb0, cifar100_efficient, acc=True)
        norm_acc_list_c100_eb0.append(acc)

print(norm_acc_list_c100_eb0)